In [1]:
import duckdb
import time
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

/home/newuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
con = duckdb.connect(database="tpch_sf1.db")

In [11]:
#2D
query_template = """select s_name, s_address
from supplier,nation
where
	s_suppkey in (
		select ps_suppkey
		from partsupp
		where
			ps_partkey in (
				select p_partkey
				from part
				where p_name like 'navajo%'
			)
			and ps_availqty > (
				select 0.5 * sum(l_quantity)
				from lineitem
				where
					l_partkey = ps_partkey
					and l_suppkey = ps_suppkey
					and l_shipdate >= '$shipdate1'
					and l_shipdate < '$shipdate2'
			)
	)
	and s_nationkey = n_nationkey and n_name = 'ETHIOPIA'
order by s_name;"""


In [4]:
min_shipdate = con.sql("SELECT MIN(l_shipdate) FROM lineitem").fetchone()[0]
max_shipdate = con.sql("SELECT MAX(l_shipdate) FROM lineitem").fetchone()[0]
(min_shipdate, max_shipdate)

(datetime.date(1992, 1, 2), datetime.date(1998, 12, 1))

In [17]:
table = []

# Loop over the date range for `receiptdate`
for shipdate1 in tqdm(list(pd.date_range(min_shipdate, max_shipdate, freq='30D'))):
    # Loop over the range for `receiptdate`
    for shipdate2 in tqdm(list(pd.date_range(min_shipdate, max_shipdate, freq='30D')), leave=False):
            params = {'shipdate1': shipdate1, 'shipdate2': shipdate2}
            for key, val in params.items():
                query_template = query_template.replace(f"${key}", str(val))
            start = time.time()
            #print(query_template)
            res = con.sql(query_template)
            elapsed = time.time() - start
            params['elapsed'] = elapsed
            table.append(params)

table = pd.DataFrame(table)
table

100%|██████████| 85/85 [00:02<00:00, 39.63it/s]


,shipdate1,shipdate2,elapsed
0,1992-01-02,1992-01-02,0.000648
1,1992-01-02,1992-02-01,0.000348
2,1992-01-02,1992-03-02,0.000264
3,1992-01-02,1992-04-01,0.000251
4,1992-01-02,1992-05-01,0.000246
...,...,...,...
7220,1998-11-26,1998-07-29,0.000250
7221,1998-11-26,1998-08-28,0.000241
7222,1998-11-26,1998-09-27,0.000262
7223,1998-11-26,1998-10-27,0.000236


In [18]:
con.close()